In [1]:
def task2():
    import tensorflow as tf
    from hops import tensorboard
    from hops import hdfs
    
    from tensorflow.examples.tutorials.mnist import input_data
    
#     fashion_mnist = input_data.read_data_sets('input/fashion', one_hot=True)
    fashion_mnist = input_data.read_data_sets('data/fashion', source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/')
    
    # Helpers
    def weight_var(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_var(shape, value=0):
        initial = tf.constant(value, shape=shape)
        return tf.Variable(initial)
    
    def bias_var_z(shape):
        return tf.Variable(tf.zeros(shape))
    
    def layer(tensor, in_dim, out_dim, name, activation=tf.nn.sigmoid):
        weights = weight_var([in_dim, out_dim])
        biases = bias_var_z([out_dim])
        pre = tf.matmul(tensor, weights) + biases
        post = activation(pre)
        tf.summary.histogram('activations', post)
        return post
    
    # Hardcoded params
    num_ch = 1
    num_classes = 10
    image_height = image_width = 28
    layer_widths = [200, 100, 60, 30, 10]
    
    # 1. Define variables and placeholders
    X = tf.placeholder(tf.float32, shape=[None, image_height, image_width, num_ch])
    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(X, [-1, 28, 28, 1])
        tf.summary.image('input', image_shaped_input, 10)

    
    Y_ = tf.placeholder(tf.float32, shape=[None, 10])
    XX = tf.reshape(X, [-1, image_height * image_width])
    
    HSig1 = layer(XX, 784, layer_widths[0], 'sigmoid-1', tf.nn.sigmoid)
    HSig2 = layer(HSig1, layer_widths[0], layer_widths[1], 'sigmoid-2', tf.nn.sigmoid)
    HSig3 = layer(HSig2, layer_widths[1], layer_widths[2], 'sigmoid-3', tf.nn.sigmoid)
    HSig4 = layer(HSig3, layer_widths[2], layer_widths[3], 'sigmoid-4', tf.nn.sigmoid)
    
    Y = layer(HSig4, layer_widths[3], layer_widths[4], 'logits', tf.identity)

    cross_entropy_mean = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_))
    tf.summary.scalar('cross_entropy', cross_entropy_mean)
        
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
    with tf.name_scope('train'):
        with tf.name_scope('gradient_descent'):
            train_step_gd = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy_mean)
        with tf.name_scope('adam_optimizer'):
            train_step_adam = tf.train.AdamOptimizer(0.005).minimize(cross_entropy_mean)

    
    # Define accuracy
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction =  tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    sess = tf.Session()
    logdir = tensorboard.logdir()
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(logdir+ '/train', sess.graph)
    test_writer = tf.summary.FileWriter(logdir + '/test')
    
    
#     init =  tf.global_variables_initializer()
    
    
    def epochs(train, test, train_step, num_epochs=100, batch_size=100):
#         sess.run(init)
        sess.run(tf.global_variables_initializer())
        accuracies = []
        losses = []
        for epoch in range(num_epochs):
            
            for it in range(100):
                batch_xs, batch_ys = train.next_batch(batch_size)
                feed_dict =  {XX: batch_xs, Y_: batch_ys}
                sess.run(train_step, feed_dict=feed_dict)

            # Compute accuracy and loss every 100 rounds
            feed_dict = {XX: test.images, Y_: test.labels}
            """
                 summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
                test_writer.add_summary(summary, i)
            """
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict)
            test_writer.add_summary(summary, epoch)
            loss = sess.run(cross_entropy_mean, feed_dict=feed_dict)
            
            accuracies.append(acc)
            losses.append(loss)
        return (accuracies, losses)


#     acc, loss = epochs(fashion_mnist.train, fashion_mnist.test, train_step_gd)
    
#     print("Loss: {}".format(loss))
#     print("Accuracy: {}".format(acc))
    train_writer.close()
    test_writer.close()
    



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1113,application_1511276242554_0449,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [ ]:
from hops import tflauncher
from hops import util

# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()

print("Logdir here XXX: ${}".format(tensorboard_logdir))

tb_hdfs_dir = tflauncher.launch(spark, task2)

tensorboard.visualize(spark, tb_hdfs_dir)